In [ ]:
import os, re
import google.generativeai as genai

os.environ["HF_TOKEN"] = ""
genai.configure(api_key="")

In [2]:
import json
from huggingface_hub import hf_hub_download

tdc_prompts_filepath = hf_hub_download(
    repo_id="google/txgemma-27b-predict",
    filename="tdc_prompts.json",
)

with open(tdc_prompts_filepath, "r") as f:
    tdc_prompts_json = json.load(f)

/usr/local/lib/python3.10/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
tdc_prompts_json["BindingDB_Patent"]

'Instructions: Answer the following question about drug target interactions.\nContext: Drug-target interaction prediction task aims to predict the interaction activity score in silico given only the accessible compound structural information and protein amino acid sequence.\nQuestion: Given the drug SMILES string and target amino acid sequence predict the normalized binding affinity from 000 to 1000, where 000 is minimum binding affinity and 1000 is maximum binding affinity.\nDrug SMILES: {Drug SMILES}\nTarget amino acid sequence: {Target amino acid sequence}\nAnswer:'

In [4]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

PREDICT_VARIANT = "9b-predict"  # @param ["2b-predict", "9b-predict", "27b-predict"]
CHAT_VARIANT = "9b-chat" # @param ["9b-chat", "27b-chat"]
USE_CHAT = True # @param {type: "boolean"}

quantization_config = BitsAndBytesConfig(load_in_4bit=True)

predict_tokenizer = AutoTokenizer.from_pretrained(f"google/txgemma-{PREDICT_VARIANT}")
predict_model = AutoModelForCausalLM.from_pretrained(
    f"google/txgemma-{PREDICT_VARIANT}",
    device_map="auto",
    quantization_config=quantization_config,
)

if USE_CHAT:
    chat_tokenizer = AutoTokenizer.from_pretrained(f"google/txgemma-{CHAT_VARIANT}")
    chat_model = AutoModelForCausalLM.from_pretrained(
        f"google/txgemma-{CHAT_VARIANT}",
        device_map="auto",
        quantization_config=quantization_config,
    )

Loading checkpoint shards: 100%|██████████| 4/4 [00:11<00:00,  2.93s/it]


In [12]:
## Example task and input
task_name = "BindingDB_Patent"
smiles = "{Drug SMILES}"
sequence = "{Target amino acid sequence}"
drug_smiles = "C[C@@H]1Cc2c([nH]c3ccccc23)[C@H](N1CC(F)(F)F)c1c(F)ccc(NCCNCCCF)c1F"
AA_sequence = "MTAEKEKKRCSSERRKEKSRDAARCRRSKETEVFYELAHQLPIPHSISSHLDKASIMRLAISFLRTRKLLTSGCVAATETTDVDRLMDSWYLKPLGGFITVVTSDGDMIFLSENINKFMGLTQVELTGHSIFDFTHPCDHEEIRENLSLKAGMGKKGKELNTERDFFMRMKCTVTNRGRTVNLKSASWKVLHCTGHLKVCNGCPARVLCGFKEPPLTCVVMMCEPIPHPSNIDTPLDSKAFLSRHSMDMKFTYCDDRVTELMGYSPEDLLGRSAYDFYHALDSDNVTKSHQNLCTKGQAVSGQYRMLAKNGGYVWVETQGTVIYNSRNSQPQCIVCVNYVLSDVEEKSMIFSMDQTESLFKPHNLNSFFSPSKRSLGSDQSEALFTKLKEEPEDLTQLAPTPGDTIISLDFGQPQYEEHPMYSKVSSVAPPVSHSIHDGHKASYAGDMPKMAATFSVPQAPPPSSATPSLSSCSTPSSPGDYYTPVDSDLKVELTEKLFSLDTQETKASCNQENDLSDLDLETLAPYIPMDGEDFQLNPICQEEPASEIGGLVTNQQSFSNITSLFQPLGSSSAAHFQPNMSSGGDKKSISGGSVGSWPSIPCSRGPMQMPPYHDPASTPLSSMGGRQNLQWPPDPPLPSKAGMMDPLAAKRSCQTMPANRMPLYLQRPVENFVQNYRDMSPARLALTNGFKRSFTQMTMGESPPTKSQQTLWKRLRNESCAVMDRKSLSTSALSDKGMAHNRGMDHQHRKTQYSGNQTGQAAKCYREQCCNYREFSMQPSSKMDGIASRLIGPSFETYSLPELTRYDCEVNVPLQGNLHLLQGSDLLRALDQST"
TDC_PROMPT = tdc_prompts_json[task_name].replace(smiles, drug_smiles).replace(sequence, AA_sequence)

def txgemma_predict(prompt):
    input_ids = predict_tokenizer(prompt, return_tensors="pt").to("cuda")
    outputs = predict_model.generate(**input_ids, max_new_tokens=8)
    return predict_tokenizer.decode(outputs[0], skip_special_tokens=True)

def txgemma_chat(prompt):
    input_ids = chat_tokenizer(prompt, return_tensors="pt").to("cuda")
    outputs = chat_model.generate(**input_ids, max_new_tokens=32)
    return chat_tokenizer.decode(outputs[0], skip_special_tokens=True)

print(f"Prediction model response: {txgemma_predict(TDC_PROMPT)}")
if USE_CHAT: print(f"Chat model response: {txgemma_chat(TDC_PROMPT)}")

Prediction model response: Instructions: Answer the following question about drug target interactions.
Context: Drug-target interaction prediction task aims to predict the interaction activity score in silico given only the accessible compound structural information and protein amino acid sequence.
Question: Given the drug SMILES string and target amino acid sequence predict the normalized binding affinity from 000 to 1000, where 000 is minimum binding affinity and 1000 is maximum binding affinity.
Drug SMILES: C[C@@H]1Cc2c([nH]c3ccccc23)[C@H](N1CC(F)(F)F)c1c(F)ccc(NCCNCCCF)c1F
Target amino acid sequence: MTAEKEKKRCSSERRKEKSRDAARCRRSKETEVFYELAHQLPIPHSISSHLDKASIMRLAISFLRTRKLLTSGCVAATETTDVDRLMDSWYLKPLGGFITVVTSDGDMIFLSENINKFMGLTQVELTGHSIFDFTHPCDHEEIRENLSLKAGMGKKGKELNTERDFFMRMKCTVTNRGRTVNLKSASWKVLHCTGHLKVCNGCPARVLCGFKEPPLTCVVMMCEPIPHPSNIDTPLDSKAFLSRHSMDMKFTYCDDRVTELMGYSPEDLLGRSAYDFYHALDSDNVTKSHQNLCTKGQAVSGQYRMLAKNGGYVWVETQGTVIYNSRNSQPQCIVCVNYVLSDVEEKSMIFSMDQTESLFKPHNLNSFFSPSKRSLGSDQSEALFTK

# Tool to allow our Agentic-Tx to ask TxGemma therapeutically relevant questions

In [8]:
# This will allow us to extract content from inside of ticks
def extract_prompt(text, word):
    code_block_pattern = rf"```{word}(.*?)```"
    code_blocks = re.findall(code_block_pattern, text, re.DOTALL)
    extracted_code = "\n".join(code_blocks).strip()
    return extracted_code

# This class will allow us to inferface with TxGemma
class TxGemmaChatTool:
    def __init__(self):
      self.tool_name = "Chat Tool"

    def use_tool(self, question):
        # Here, we are submitting a question to TxGemma
        response = txgemma_chat(question)
        return response

    def tool_is_used(self, query):
        # This just checks to see if the tool call was evoked
        return "```TxGemmaChat" in query

    def process_query(self, query):
        # Here, we clean to query to remove the tool call
        return extract_prompt(query, word="TxGemmaChat")

    def instructions(self):
        # Here, we are **very** descriptively explaining how the tool works to the agent
        # This will be useful later on
        return (
            "=== Therapeutic Chat Tool Instructions ===\n"
            "### What This Tool Does\n"
            "The Therapeutic Chat Tool allows you to chat with a knowledgeable large language model named TxGemma trained on many therapeutics datasets."
            "### When and Why You Should Use It\n"
            "- If you have therapeutics related questions that you would benefit from asking TxGemma from.\n"
            "### How to Use It\n"
            "Format your query with triple backticks (```), and start with `TxGemmaChat`. Then on a new line:\n"
            "1) **Any question you would like to ask**\n\n"
            "Example:\n"
            "```TxGemmaChat\n"
            "What is a common drug used to treat ovarian cancer?\n"
            "```\n")
     

# Making a TxGemma prediction

In [24]:
# This class will allow us to predict binding affinity using TxGemma
class BindingPred:
    def __init__(self):
      self.tool_name = "Binding Affinity Prediction"

    def use_tool(self, smiles_string, AA_sequence):
        # Here, we are submitting the smiles to TxGemma, and returning the response
        prediction = txgemma_predict(tdc_prompts_json["BindingDB_Patent"].replace("{Drug SMILES}", smiles_string).replace("{Target amino acid sequence}", AA_sequence))
        match = re.search(r"Answer:\s*([0-9]+(?:\.[0-9]+)?)", prediction)
        score = float(match.group(1))
        if 650 <= score <= 1000:
            return f"{smiles_string} has a high binding affinity score of {score/1000:.1%}!"
        else:
            return f"{smiles_string} has a low binding affinity score of {score/1000:.1%}"

    def tool_is_used(self, query):
        # This just checks to see if the tool call was evoked
        return "```BindingAffPred" in query

    def process_query(self, query):
        # Here, we clean to query to remove the tool call
        return extract_prompt(query, word="BindingAffPred")

    def instructions(self):
        # Here, we are explaining how the tool works to the agent
        return (
        "=== Binding Affinity Prediction Instructions ===\n"
        "The Binding Affinity Prediction Tool computes an in silico interaction score\n"
        "between a small-molecule (SMILES) and a protein sequence (amino acids).\n\n"

        "To use this tool, invoke it exactly like this:\n"
        "```BindingAffPred\n"
        "{Drug SMILES}\n"
        "{Target amino acid sequence}\n"
        "```\n\n"

        "• **Keyword**: `BindingAffPred` (must match exactly).\n"
        "• **Line 2**: the SMILES string of your ligand.\n"
        "• **Line 3**: the amino-acid sequence of your target protein.\n\n"

        "**Example:**\n"
        "```BindingAffPred\n"
        "CC(=O)Oc1ccccc1C(=O)O\n"
        "MTEITAAMVKELRESTGAGMMDCKNALSETQHEKATEPTLQA" 
        "```\n"
        "This will return a binding-affinity score for that ligand–protein pair.\n")

In [30]:
bindPred = BindingPred()
prediction_belzutifan = bindPred.use_tool("COc1c(F)cccc1C1=C(c2ccc(O[C@H]3CCN(CCCF)C3)cc2)c2ccc(O)cc2CCC1", AA_sequence)
print(prediction_belzutifan)

COc1c(F)cccc1C1=C(c2ccc(O[C@H]3CCN(CCCF)C3)cc2)c2ccc(O)cc2CCC1 has a high binding affinity score of 68.1%!


# PubMed search tool

In [31]:
! pip install --upgrade --quiet biopython

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [32]:
from Bio import Medline, Entrez

# This class will allow us to interface with PubMed
class PubMedSearch:
    def __init__(self):
      self.tool_name = "PubMed Search"

    def tool_is_used(self, query: str):
        # This just checks to see if the tool call was evoked
        return "```PubMedSearch" in query

    def process_query(self, query: str):
        # Here, we clean to query to remove the tool call
        search_text = extract_prompt(query, word="PubMedSearch")
        return search_text.strip()

    def use_tool(self, search_text):
        # Here, we are searching through PubMed and returning relevant articles
        pmids = list()
        handle = Entrez.esearch(db="pubmed", sort="relevance", term=search_text, retmax=3)
        record = Entrez.read(handle)
        pmids = record.get("IdList", [])
        handle.close()

        if not pmids:
            return f"No PubMed articles found for '{search_text}' Please try a simpler search query."

        fetch_handle = Entrez.efetch(db="pubmed", id=",".join(pmids), rettype="medline", retmode="text")
        records = list(Medline.parse(fetch_handle))
        fetch_handle.close()

        result_str = f"=== PubMed Search Results for: '{search_text}' ===\n"
        for i, record in enumerate(records, start=1):
            pmid = record.get("PMID", "N/A")
            title = record.get("TI", "No title available")
            abstract = record.get("AB", "No abstract available")
            journal = record.get("JT", "No journal info")
            pub_date = record.get("DP", "No date info")
            authors = record.get("AU", [])
            authors_str = ", ".join(authors[:3])
            result_str += (
                f"\n--- Article #{i} ---\n"
                f"PMID: {pmid}\n"
                f"Title: {title}\n"
                f"Authors: {authors_str}\n"
                f"Journal: {journal}\n"
                f"Publication Date: {pub_date}\n"
                f"Abstract: {abstract}\n")
        return f"Query: {search_text}\nResults: {result_str}"

    def instructions(self):
        # Here, we are explaining how the tool works to the agent
        return (
            f"{'@' * 10}\n@@@ PubMed Search Tool Instructions @@@\n\n"
            "### What This Tool Does\n"
            "The PubMed Search Tool queries the NCBI Entrez API (PubMed) for a given search phrase, "
            "and retrieves metadata for a few of the top articles (PMID, title, authors, journal, date, abstract).\n\n"
            "### When / Why You Should Use It\n"
            "- To find **scientific literature** references on a specific biomedical topic.\n"
            "- To retrieve **abstracts, titles, authors**, and other metadata.\n\n"
            "### Query Format\n"
            "Wrap your request with triple backticks, starting with `PubMedSearch`. For example:\n\n"
            "```PubMedSearch\ncancer immunotherapy\n```\n\n"
            "### Example\n"
            "```PubMedSearch\nmachine learning in drug discovery\n```\n"
            "- This will search PubMed for articles related to 'machine learning in drug discovery', "
            "fetch up to 3 PMIDs, and return their titles, abstracts, etc.\n\n")

# Wrapping it all together

### Creating a tool manager

In [35]:
# The tool manager will hold all of the tools, and provide an interface for the agent
class ToolManager:
    def __init__(self, toolset):
        self.toolset = toolset

    def tool_prompt(self):
        # This will let the agent know what tools it has access to
        tool_names = ", ".join([tool.tool_name for tool in self.toolset])
        return f"You have access to the following tools: {tool_names}\n{self.tool_instructions()}. You can only use one tool at a time. These are the only tools you have access to nothing else."

    def tool_instructions(self):
        # This allows the agent to know how to use the tools
        tool_instr = "\n".join([tool.instructions() for tool in self.toolset])
        return f"The following is a set of instructions on how to use each tool.\n{tool_instr}"

    def use_tool(self, query):
        # This will iterate through all of the tools
        # and find the correct tool that the agent requested
        for tool in self.toolset:
            if tool.tool_is_used(query):
                # use the tool and return the output
                return tool.use_tool(tool.process_query(query))
        return f"No tool match for search: {query}"

if USE_CHAT:
    tools = ToolManager([TxGemmaChatTool(), BindingPred(), PubMedSearch()])
else:
    tools = ToolManager([BindingPred(), PubMedSearch()])

### Creating a Gemini inference tool

In [45]:
def inference_gemini(prompt, system_prompt, model_str):
  # Check to see that our model string matches
  if model_str == "gemini-2.0-pro":
    model = genai.GenerativeModel(model_name="gemini-2.0-pro-exp-02-05", system_instruction=system_prompt)
    response = model.generate_content(prompt)
    answer = response.text
  return answer

# Creating a therapeutics agent

In [48]:
# This class defines our Agentic-Tx, wrapping together all of our tools and the orchestrator
class AgenticTx:
  def __init__(self, tool_manager, model_str, num_steps=5):
    self.curr_steps = 0
    self.num_steps = num_steps
    self.model_str = model_str
    self.tool_manager = tool_manager
    self.thoughts = list()
    self.actions  = list()
    self.observations = list()

  def reset(self):
    # Reset the number of steps taken
    self.curr_steps = 0

  def system_prompt(self, use_tools=True):
    # These are the system instructions for AgenticTx
    role_prompt = "You are an expert therapeutic agent. You answer accurately and thoroughly."
    prev_actions = f"You can perform a maximum of {self.num_steps} actions. You have performed {self.curr_steps} and have {self.num_steps - self.curr_steps - 1} left."
    if use_tools: tool_prompt = "You can use tools to solve problems and answer questions. " + self.tool_manager.tool_prompt()
    else: tool_prompt = "You cannot use any tools right now."
    return f"{role_prompt} {prev_actions} {tool_prompt}"

  def prior_information(self, query):
      info_txt = f"Question: {query}\n" if query is not None else ""
      for _i in range(self.curr_steps):
          info_txt += f"### Thought {_i + 1}: {self.thoughts[_i]}\n"
          info_txt += f"### Action {_i + 1}: {self.actions[_i]}\n"
          info_txt += f"### Observation {_i + 1}: {self.observations[_i]}\n\n"
          info_txt += "@"*20
      return info_txt

  def step(self, question):
    for _i in range(self.num_steps):
      if self.curr_steps == self.num_steps-1:
        return inference_gemini(
            model_str=self.model_str,
            prompt=f"{self.prior_information(question)}\nYou must now provide an answer to this question {question}",
            system_prompt=self.system_prompt(use_tools=False))
      else:
        # Provide a thought step, planning for the model
        thought = inference_gemini(
            model_str=self.model_str,
            prompt=f"{self.prior_information(question)}\nYou cannot currently use tools but you can think about the problem and what tools you want to use. This was the question, think about plans for how to use tools to answer this {question}. Let's think step by step (respond with only 1-2 sentences).\nThought: ",
            system_prompt=self.system_prompt(use_tools=False))
        # Provide a took action for the model
        action = inference_gemini(
            model_str=self.model_str,
            prompt=f"{self.prior_information(question)}\n{thought}\nNow you must use tools to answer the following user query [{question}], closely following the tool instructions. Tool",
            system_prompt=self.system_prompt(use_tools=True))
        obs = self.tool_manager.use_tool(action)

        print("Thought:", thought)
        print("Action:",  action)
        print("Observation:",  obs)

        self.thoughts.append(thought)
        self.actions.append(action)
        self.observations.append(obs)

        self.curr_steps += 1


agentictx = AgenticTx(tool_manager=tools, model_str="gemini-2.0-pro")
# It should select CS(=O)(=O)C1=C2[C@@H]([C@@H]([C@@H](C2=C(C=C1)OC3=CC(=CC(=C3)C#N)F)F)F)O because it is a known HIF-2a binder
response = agentictx.step("Which of the following drugs is preferred for HIF-2a inhibition? 1. CS(=O)(=O)C1=C2[C@@H]([C@@H]([C@@H](C2=C(C=C1)OC3=CC(=CC(=C3)C#N)F)F)F)O or 2. COc1c(F)cccc1C1=C(c2ccc(O[C@H]3CCN(CCCF)C3)cc2)c2ccc(O)cc2CCC1")
print("\nFinal Response:", response)

Thought: Okay, I will first try to identify the chemical structures from the provided SMILES strings. Then, I will research their known activities as HIF-2a inhibitors to determine which, if either, is preferred.
Action: ```TxGemmaChat
Are the following SMILES strings known HIF-2a inhibitors? If so, what are their common names?
1. CS(=O)(=O)C1=C2[C@@H]([C@@H]([C@@H](C2=C(C=C1)OC3=CC(=CC(=C3)C#N)F)F)F)O
2. COc1c(F)cccc1C1=C(c2ccc(O[C@H]3CCN(CCCF)C3)cc2)c2ccc(O)cc2CCC1
```
Observation: Are the following SMILES strings known HIF-2a inhibitors? If so, what are their common names?
1. CS(=O)(=O)C1=C2[C@@H]([C@@H]([C@@H](C2=C(C=C1)OC3=CC(=CC(=C3)C#N)F)F)F)O
2. COc1c(F)cccc1C1=C(c2ccc(O[C@H]3CCN(CCCF)C3)cc2)c2ccc(O)cc2CCC1=O
3. C1(P(C2C=CC=CC=2)C2C=CC=CC=2)C=
Thought: Okay, I will first try to identify the chemical structures from the provided SMILES strings. Then, I will research their known activities as HIF-2a inhibitors, including potency, selectivity, and clinical status, to determine whi